## Capstone Jupyter Notebook
__Author: Sam Steffes__
<br>*Date: 08/18/2020*

This notebook will be used for the Coursera IBM Data Science Capstone project. This project will use data from previous vehicle collisions to try and build a machine learning model that predicts the probability and severity of accidents based on given conditions.

In [1]:
#import some required libraries
import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
